# 환경설정

In [1]:
import random
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import json
from sklearn.model_selection import train_test_split
from copy import deepcopy


In [2]:
# 재현성
def fix_seed(random_seed):
    np.random.seed(random_seed)
    random.seed(random_seed)

fix_seed(42)

## random train/val split

In [5]:
"""
idx를 받아 train/valid split annotation json 생성하는 함수

idx: annotations json 리스트에서의 index
id : Image/Annotation 식별 id
"""

def generate_split(train_idx, val_idx):
    train_data = deepcopy(data)
    val_data = deepcopy(data)
    
    df_imgs = pd.DataFrame(data['images']).transpose()
    
    train_data['images'] = df_imgs.iloc[list(train_idx)].to_dict('index')
    val_data['images'] = df_imgs.iloc[list(val_idx)].to_dict('index')
    
    return train_data, val_data

annotation = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'

with open(annotation) as f: data = json.load(f)

dataset_len = len(data['images'])
percent = 0.1 # test/valid split 비율

dummy_idx = {x for x in range(dataset_len)} # 데이터셋 전체 길이의 더미 인덱스
val_idx = set(random.sample(dummy_idx, int(dataset_len*percent))) # 랜덤으로 추출된 validation idx
train_idx = dummy_idx - val_idx # 랜덤으로 추출된 train idx

assert len(val_idx & train_idx) == 0 ,"error: 중복된 값이 있음"
print(f"Total dataset 크기: {len(dummy_idx)}\n Train dataset 크기: {len(train_idx)}\n Val dataset 크기: {len(val_idx)}")

# train_idx, val_idx 는 annotation의 idx임. (!= id)
train_data, val_data = generate_split(train_idx, val_idx)

with open(f'/opt/ml/input/data/ICDAR17_Korean/random_split_ufo/train.json', 'w') as f:
    json.dump(train_data, f, indent=2)

with open(f'/opt/ml/input/data/ICDAR17_Korean/random_split_ufo/val.json', 'w') as f:
    json.dump(val_data, f, indent=2)


Total dataset 크기: 536
 Train dataset 크기: 483
 Val dataset 크기: 53
